In [2]:
# !wget https://object.pouta.csc.fi/OPUS-Ubuntu/v14.10/raw/en.zip
# !wget https://object.pouta.csc.fi/OPUS-Ubuntu/v14.10/raw/ms.zip
# !unzip -o en.zip
# !unzip -o ms.zip

In [4]:
# !wget https://object.pouta.csc.fi/OPUS-Ubuntu/v14.10/xml/en-ms.xml.gz
# !gzip -d en-ms.xml.gz

In [5]:
from bs4 import BeautifulSoup
from tqdm import tqdm

with open('en-ms.xml') as fopen:
    xml = fopen.read().split('\n')
    
mapping, ids = {}, []
enable = False
for row in tqdm(xml):
    if '<linkGrp targType' in row:
        s = BeautifulSoup(row)
        r = 'Ubuntu/raw/' + s.findAll('linkgrp')[0].get('fromdoc').replace('.gz','')
        l = 'Ubuntu/raw/' + s.findAll('linkgrp')[0].get('todoc').replace('.gz','')
        enable = True
    if '<link ' in row and enable:
        s = BeautifulSoup(row)
        t = s.findAll('link')[0].get('xtargets')
        t = list(filter(None, t.split(';')))
        if len(t) != 2:
            continue
        ids.append([t[0].split(), t[1].split()])
    if '</linkGrp>' in row:
        label = f'{l} <> {r}'
        mapping[label] = ids
        enable = False
        ids = []

100%|██████████| 121906/121906 [00:28<00:00, 4314.15it/s]


In [7]:
results = []
for file in tqdm(mapping.keys()):
    l, r = file.split(' <> ')
    ids = mapping[file]
    try:
        with open(l) as fopen:
            left = fopen.read()
        soup_left = BeautifulSoup(left, 'html.parser')
        with open(r) as fopen:
            right = fopen.read()
        soup_right = BeautifulSoup(right, 'html.parser')
        left_dict = {}
        for s in soup_left.find_all('s'):
            i = s.get('id')
            text = s.get_text()
            left_dict[i] = text
        right_dict = {}
        for s in soup_right.find_all('s'):
            i = s.get('id')
            text = s.get_text()
            right_dict[i] = text
        combined = []
        for i in ids:
            try:
                r = ' '.join([right_dict[k] for k in i[0]])
                l = ' '.join([left_dict[k] for k in i[1]])
                combined.append((l, r))
            except:
                pass
        results.extend(combined)
    except Exception as e:
        print(e)
        pass

100%|██████████| 451/451 [00:21<00:00, 21.41it/s]


In [8]:
len(results)

120097

In [9]:
results[:20]

[('Sertakan Gmail, Google Docs, Google+, YouTube dan Picasa',
  'Includes Gmail, Google Docs, Google+, YouTube and Picasa'),
 ('Ubah data pengguna yang anda miliki', 'Change your own user data'),
 ('Pengesahihan diperlukan untuk ubah data pengguna yang anda miliki',
  'Authentication is required to change your own user data'),
 ('Urus akaun pengguna', 'Manage user accounts'),
 ('Pengesahihan diperlukan untuk ubah data pengguna',
  'Authentication is required to change user data'),
 ('Ubah konfigurasi skrin daftar masuk',
  'Change the login screen configuration'),
 ('Pengesahihan diperlukan untuk ubah konfigurasi skrin daftar masuk',
  'Authentication is required to change the login screen configuration'),
 ('Outputkan maklumat versi dan keluar',
  'Output version information and exit'),
 ('Ganti kejadian sedia ada', 'Replace existing instance'),
 ('Benarkan kod penyahpepijat', 'Enable debugging code'),
 ('Menyediakan antaramuka D-Bus untuk pertanyaan dan\nmanipulasikan maklumat akaun 

In [10]:
import json

with open('ubuntu-ms-en.json', 'w') as fopen:
    json.dump(results, fopen)

In [11]:
!rm -rf Ubuntu
!rm -rf ms.zip en.zip
!rm -rf ms.zip en.zip en-ms.xml.gz en-ms.xml